In [1]:
import h5py
import numpy as np
import os
import matplotlib.pyplot as plt
import os
import tensorflow as tf

2025-07-21 05:58:45.128401: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753102725.141823 1321413 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753102725.145968 1321413 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753102725.157770 1321413 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753102725.157782 1321413 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753102725.157784 1321413 computation_placer.cc:177] computation placer alr

In [2]:
# minorized reference
with h5py.File('/global/u2/k/kberard/SCGSR/Research/Diamond/Data/density_tot_ref.h5', 'r') as file:
    #print("Keys: %s" % file.keys())
    ref_d = file['density'][:]
#print(ref_d)
print(ref_d.shape)




(64, 64, 64)


In [3]:
####################################################################################################################################################
def stochastic_density(d,N):
    # poisson model
    #  accurate and fast for all values of N
    # N  = number of MC samples
    assert isinstance(d,np.ndarray)
    assert isinstance(N,(int,float,np.int64,np.float64))
    assert N>0
    ds = np.random.poisson(N*d)/N
    ds*= d.sum()/ds.sum()
    return ds
#end def stochastic_density

####################################################################################################################################################

In [4]:
import tensorflow as tf

def jensen_shannon_divergence_loss(y_true, y_pred):
    y_t = tf.cast(y_true, tf.float32)
    y_p = tf.cast(y_pred, tf.float32)

    y_t = tf.reshape(y_t, [tf.shape(y_t)[0], -1])
    y_p = tf.reshape(y_p, [tf.shape(y_p)[0], -1])

    y_t /= tf.reduce_sum(y_t, axis=1, keepdims=True) + 1e-8
    y_p /= tf.reduce_sum(y_p, axis=1, keepdims=True) + 1e-8

    m = 0.5 * (y_t + y_p)

    kl_true = tf.reduce_sum(y_t * tf.math.log((y_t + 1e-8) / (m + 1e-8)), axis=1)
    kl_pred = tf.reduce_sum(y_p * tf.math.log((y_p + 1e-8) / (m + 1e-8)), axis=1)

    jsd = 0.5 * (kl_true + kl_pred)

    return tf.reduce_mean(jsd)


In [5]:
def generate_single_2d_density(ref_d, noise_level):
    noisy_3d = stochastic_density(ref_d, noise_level)  # shape: (64, 64, 64)
    return noisy_3d

# Noise levels to randomly draw from
noise_levels = [10000 * (j + 1) for j in range(5)]  # [10000, 20000, ..., 50000]

# Sizes
num_train = 5000
num_val = 1000
num_test = 500

# Generate train set with mixed noise levels
x_train = np.zeros((num_train, 64, 64,64))
for i in range(num_train):
    noise_level = np.random.choice(noise_levels)
    x_train[i] = generate_single_2d_density(ref_d, noise_level)

# Generate val set with mixed noise levels
x_val = np.zeros((num_val, 64, 64,64))
for i in range(num_val):
    noise_level = np.random.choice(noise_levels)
    x_val[i] = generate_single_2d_density(ref_d, noise_level)



print("Train, val, generated")


Train, val, generated


In [6]:
# Generate test set with fixed noise level
x_test = np.zeros((num_test, 64, 64, 64))
for i in range(num_test):
    x_test[i] = generate_single_2d_density(ref_d, 100000)
print("test generated")

test generated


In [7]:
y_test_3d = np.zeros((x_test.shape[0],64,64,64))
y_train_3d = np.zeros((x_train.shape[0],64,64,64))
y_val_3d = np.zeros((x_val.shape[0],64,64,64))
for i in range (y_test_3d.shape[0]):
    y_test_3d[i]=stochastic_density(ref_d,1000000)
y_test = y_test_3d
for i in range (y_train_3d.shape[0]):
    y_train_3d[i]=stochastic_density(ref_d,1000000)
y_train = y_train_3d

for i in range (y_val_3d.shape[0]):
    y_val_3d[i]=stochastic_density(ref_d,1000000)
y_val = y_val_3d
print("here")

here


In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, regularizers
import matplotlib.pyplot as plt

def renormalize_sum_to_eight(x):
    total = tf.reduce_sum(x, axis=[1, 2, 3, 4], keepdims=True)
    return x / (total + 1e-8) * 8.0

def jensen_shannon_divergence_loss(y_true, y_pred):
    y_t = tf.cast(y_true, tf.float32)
    y_p = tf.cast(y_pred, tf.float32)
    y_t = tf.reshape(y_t, [tf.shape(y_t)[0], -1])
    y_p = tf.reshape(y_p, [tf.shape(y_p)[0], -1])
    y_t /= tf.reduce_sum(y_t, axis=1, keepdims=True) + 1e-8
    y_p /= tf.reduce_sum(y_p, axis=1, keepdims=True) + 1e-8
    m = 0.5 * (y_t + y_p)
    kl_true = tf.reduce_sum(y_t * tf.math.log((y_t + 1e-8) / (m + 1e-8)), axis=1)
    kl_pred = tf.reduce_sum(y_p * tf.math.log((y_p + 1e-8) / (m + 1e-8)), axis=1)
    return tf.reduce_mean(0.5 * (kl_true + kl_pred))

# --- Fully Connected Autoencoder ---
def create_dense_autoencoder_with_pooling(input_shape=(64, 64, 64, 1)):
    input_layer = layers.Input(shape=input_shape)

    # Downsample
    x = layers.AveragePooling3D(pool_size=(2, 2, 2))(input_layer)  # → 32³
    x = layers.AveragePooling3D(pool_size=(2, 2, 2))(x) 
    x = layers.AveragePooling3D(pool_size=(2, 2, 2))(x) # → 16³
    x = layers.AveragePooling3D(pool_size=(2, 2, 2))(x) 
    x = layers.Flatten()(x)

    # Dense bottleneck
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(128, activation='relu')(x)

    # Decoder
    x = layers.Dense(16*16*16, activation='relu')(x)
    x = layers.Reshape((16, 16, 16, 1))(x)
    x = layers.UpSampling3D((2, 2, 2))(x)
    x = layers.UpSampling3D((2, 2, 2))(x)

    # Final output
    x = layers.Conv3D(1, (3, 3, 3), activation='relu', padding='same')(x)
    output_layer = layers.Lambda(renormalize_sum_to_eight)(x)

    return models.Model(inputs=input_layer, outputs=output_layer)


# --- Training Pipeline ---
if __name__ == '__main__':
    # Your data should be loaded like this:
    # x_train, y_train = shape (N, 64, 64, 64, 1)
    # Ensure all are normalized and dtype float32
    input_shape = (64, 64, 64, 1)
    model = create_dense_autoencoder(input_shape=input_shape)

    model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss=jensen_shannon_divergence_loss)

    # Callbacks
    early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, min_lr=1e-6)
    callbacks_list = [early_stop, lr_scheduler]

    # Train
    model.fit(x_train, y_train,
              validation_data=(x_val, y_val),
              epochs=5,
              batch_size=4,
              callbacks=callbacks_list,
              verbose=1)

    # Predict and plot
    i = 0
    denoised = model.predict(x_test[i:i+1])[0, ..., 0]
    noisy_input = x_test[i, ..., 0]
    true_clean = y_test[i, ..., 0]

    z_mid = 64 // 2
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.imshow(noisy_input[:, :, z_mid], cmap='viridis')
    plt.title("Noisy Input")
    plt.colorbar()

    plt.subplot(1, 3, 2)
    plt.imshow(denoised[:, :, z_mid], cmap='viridis')
    plt.title("Denoised Output")
    plt.colorbar()

    plt.subplot(1, 3, 3)
    plt.imshow(true_clean[:, :, z_mid], cmap='viridis')
    plt.title("Clean Target")
    plt.colorbar()

    plt.tight_layout()
    plt.show()


2025-07-21 06:14:36.163234: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.88GiB (rounded to 5242880000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-07-21 06:14:36.163279: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2025-07-21 06:14:36.163292: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 50, Chunks in use: 50. 12.5KiB allocated for chunks. 12.5KiB in use in bin. 2.8KiB client-requested in use in bin.
2025-07-21 06:14:36.163299: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (512): 	Total Chunks: 10, Chunks in use: 10. 5.0KiB allocated for chunks. 5.0KiB in use in bin. 5.0KiB client-requested in use in bin.
2025-

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

B in use in bin. 0B client-requested in use in bin.
2025-07-21 06:14:36.163347: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (131072): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2025-07-21 06:14:36.163353: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (262144): 	Total Chunks: 12, Chunks in use: 10. 3.12MiB allocated for chunks. 2.62MiB in use in bin. 2.50MiB client-requested in use in bin.
2025-07-21 06:14:36.163358: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (524288): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2025-07-21 06:14:36.163364: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (1048576): 	Total Chunks: 6, Chunks in use: 6. 6.35MiB allocated for chunks. 6.35MiB in use in bin. 6.00MiB client-requested in use in bin.
2025-07-21 06:14:36.163370: I external/loca

In [32]:
print(x_test.shape,"Shape of test set")
# must beat the model predicting the average
test_avg = np.mean(x_test,axis=0)
print(test_avg.shape, "Shape of average of test set")
print(ref_d.shape,"reference shape")
print("2 norm difference")
print("Mean vs DFT 2 norm diff")
print(np.linalg.norm(test_avg[:,:,:,0]-ref_d))

print("prediction of one test sample vs DFT")
print(x_test[0:0+1].shape,"test_shape")
print(cae.predict(x_test[0:0+1])[0,:,:,:,0].shape,ref_d.shape)
print(np.linalg.norm(cae.predict(x_test[0:0+1])[0,:,:,:,0]-ref_d))
print()
print()
print("divergence difference")
print(test_avg.shape)
mean_pred = cae.predict(test_avg.reshape(1,64,64,64,1))[0,:,:,:,0]                 
print("JSD (bar{x_test},y_dft):", jensen_shannon_divergence_loss(test_avg, ref_d.copy()))
print("JSD (DAE(bar{x}),y_dft):", jensen_shannon_divergence_loss(np.squeeze(mean_pred), ref_d.copy()))
print("JSD (DAE(_i}),y_dft):", jensen_shannon_divergence_loss(cae.predict(x_test[0:0+1])[0,:,:,:,0], ref_d.copy()))


print("Working slightly better!!!!!!!!")

(500, 64, 64, 64, 1) Shape of test set
(64, 64, 64, 1) Shape of average of test set
(64, 64, 64) reference shape
2 norm difference
Mean vs DFT 2 norm diff
0.00040043202591896483
prediction of one test sample vs DFT
(1, 64, 64, 64, 1) test_shape
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
(64, 64, 64) (64, 64, 64)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
0.0003978254910755407


divergence difference
(64, 64, 64, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
JSD (bar{x_test},y_dft): tf.Tensor(9.7360404e-05, shape=(), dtype=float32)
JSD (DAE(bar{x}),y_dft): tf.Tensor(6.967109e-05, shape=(), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
JSD (DAE(_i}),y_dft): tf.Tensor(7.1515096e-05, shape=(), dtype=float32)
Working slightly better!!!!!!!!
